In [2]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjgFd71gwEv_5PiUZu9dMN1-D18xTx5DGpUtSnfokqKABd_JsCX5Fc

Successfully saved authorization token.


In [15]:
javascript_code = """
"""

In [16]:
geemap.js_snippet_to_py(javascript_code)

## Imports

In [4]:
#*** Start of imports. If edited, may not auto-convert in the playground. ***#
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
miningPermits = ee.FeatureCollection("users/raymondeah/Democratic_Republic_of_the_Congo_mining_permits")
activeMines = ee.FeatureCollection("users/raymondeah/cod_mines_curated_all_opendata_p_ipis")
minesGrouped = ee.Geometry.Polygon(
    [[[28.84879973081211, -5.016248778035446],
    [28.84879973081211, -5.037794913801396],
    [28.874162699256935, -5.037794913801396],
    [28.874162699256935, -5.016248778035446]]])

water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([28.85818756937104, -5.023243104634987]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([28.865483177891548, -5.022623221835797]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([28.866041077366646, -5.029848026118908]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([28.858101738682564, -5.027753268628657]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([28.869774712315376, -5.031964151841523]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([28.85595597147065, -5.022665972392595]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([28.86797226785737, -5.022152965525988]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([28.871555699101265, -5.020507065770843]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([28.860891236058052, -5.033631397564143]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([28.859002960911567, -5.032904649953931]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([28.852780235997017, -5.031130527378187]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([28.872027767887886, -5.034123020487134]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([28.870568646183784, -5.034849766736094]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([28.86217299426854, -5.018909704650508]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([28.852795978693162, -5.033203542518063]),
            {
              "landcover": 2,
              "system:index": "4"
            })])
#**** End of imports. If edited, may not auto-convert in the playground. ****#

#### Cloud Masking Function

In [5]:
# Function to remove cloud and snow pixels from Sentinel-2 SR image
def maskCloudAndShadowsSR(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(10)
  scl = image.select('SCL')
  shadow = scl.eq(3); # 3 = cloud shadow
  cirrus = scl.eq(10); # 10 = cirrus
  # Cloud probability less than 10% or cloud shadow classification
  mask = cloud.And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask)

#### Center the map around our chosen geometry

In [6]:
Map = geemap.Map()
congo = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))
Map.centerObject(minesGrouped, 12)

#### 2019 Median Composite

In [7]:
filtered = s2 \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
  .map(maskCloudAndShadowsSR) \
  .filter(ee.Filter.bounds(minesGrouped)) \
  .select('B.*')

composite = filtered.median().clip(minesGrouped)

rgbVis = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

Map.addLayer(composite, rgbVis, '2019 Median Composite')

#### NDVI

In [8]:
# Calculate  Normalized Difference Vegetation Index (NDVI)
# 'NIR' (B8) and 'RED' (B4)
ndvi = composite.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}
#Map.addLayer(ndvi, ndviVis, 'NDVI')

#### Mining Permits + Active Mines Dataset

In [9]:
#Map.addLayer(miningPermits, {color: 'blue'}, 'Mines')
Map.addLayer(activeMines, {'color': 'purple'}, 'Active Coltan Mines')

minesTa = miningPermits.filter(ee.Filter.stringContains('resource', 'Ta'))

#Map.addLayer(minesTa, {color: 'red'}, 'Coltan Mines')

## Classification of Change

In [11]:
training = bare.merge(water).merge(vegetation)

# Overlay the point on the image to get training data.
training = composite.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 10
})

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite.bandNames()
})

# # Classify the image.
beforeClassified = composite.classify(classifier)
Map.addLayer(beforeClassified,
  {'min': 0, 'max': 2, 'palette': ['blue', 'green', 'brown']}, 'before_classified')

# 2021
filtered = s2 \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
  .map(maskCloudAndShadowsSR) \
  .filter(ee.Filter.bounds(minesGrouped)) \
  .select('B.*')

compositeA = filtered.median().clip(minesGrouped)

Map.addLayer(compositeA, rgbVis, '2021 Median Composite')

# Classify the image.
afterClassified= compositeA.classify(classifier)
Map.addLayer(afterClassified,
  {'min': 0, 'max': 2, 'palette': ['blue', 'green', 'brown']}, 'after_classified')

# Reclassify from 0-3 to 1-4
beforeClasses = beforeClassified.remap([0, 1, 2], [1, 2, 3])
afterClasses = afterClassified.remap([0, 1, 2], [1, 2, 3])

# Show all changed areas
class3 = beforeClasses.eq(2)
notclass3 = afterClasses.eq(2).Not()
waterChanged = class3.And(notclass3)

v = beforeClasses.eq(2)
b = afterClasses.eq(3)
v_to_b = v.And(b)

Map.addLayer(waterChanged, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'NDVI_change')
Map.addLayer(v_to_b, {'min': 0, 'max': 1, 'palette': ['white', 'brown']}, 'NDVI to bare')

legend_keys = ['Water', 'Vegetation', 'Bare Soil']
legend_colors = ['#0000FF', '#00FF00', '#FF0000']

Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')
Map.centerObject(minesGrouped, 15)

Map

Map(bottom=4311906.0, center=[-5.027021909031205, 28.86148121502483], controls=(WidgetControl(options=['positi…

## Results

In [30]:
areaImage = v_to_b.multiply(ee.Image.pixelArea())

area = areaImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': minesGrouped,
  'scale': 10,
  'maxPixels': 1e10
})

net_change = ee.Number(area.get('remapped')).divide(1e6).getInfo()
print(net_change)

v_total = beforeClassified.multiply(ee.Image.pixelArea())

area2 = v_total.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': minesGrouped,
  'scale': 10,
  'maxPixels': 1e10
})

total = ee.Number(area2.get('classification')).divide(1e6).getInfo()
print(net_change / total * 100)

0.2556178112476424
4.116399416855543
